## Open data battle. Отборочный этап.
### Задача 2.

В одной компании произошла неприятность: случился потоп, в бумажных анкетах клиентов размыло некоторые строки и в 1000 анкетах пострадало поле “gender”. Конечно, их было бы можно восстановить, но в компании решили автоматизировать решение проблемы и определить пол клиентов по фотографиям. Был создан датасет gender.csv, в котором записаны признаки человека на фото и его пол.

Первые 1000 значений имеют пропуски в столбце gender.

Все столбцы с ".1" в названии дублируют предыдущие столбцы без ".1", но содержат уникальные значения

Необходимо произвести чистку данных:

Присоединить дублирующие столбцы к основным.
Столбцы 'nose_wide', 'nose_long', 'lips_thin', 'distance_nose_to_lip_long', 'long_hair' содержат информацию в условных единицах. Известно, что измерения проводились разными людьми с разными измерительными инструментами, поэтому данные в столбцах имеют разную погрешность измерений. Необходимо уменьшить влияние погрешности путем приравнивания значений диапазона [0,0.5) к 0, а [0.5, 1) —- к 1. Столбцы 'orehead_width_cm' и 'forehead_height_cm' необходимо оставить «как есть».
Оставить один из дублирующих друг друга столбцов.
Определите пол по остальным параметрам из выборки для первой тысячи значений и прикрепите его в CSV формате. Файл должен содержать только прогнозные значения в формате "Male"/"Female", в одном столбце без заголовка

Пороговое значение для данной задачи классификации считать равным 0.5.

Для успешного прохождения достаточно иметь 70% точность решения

In [157]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression as LR

In [158]:
df = pd.read_csv("gender.csv")

In [159]:
df

,Unnamed: 0,index,long_hair,forehead_width_cm,forehead_height_cm,forehead_width_mm,forehead_width_conventional_units,nose_wide,nose_long,lips_thin,...,long_hair.1,forehead_width_cm.1,forehead_height_cm.1,forehead_width_mm.1,forehead_width_conventional_units.1,nose_wide.1,nose_long.1,lips_thin.1,distance_nose_to_lip_long.1,gender.1
0,0,0,0.704275,11.8,6.1,118.0,0.025806,0.895614,0.477485,0.841261,...,0.818018,11.5,5.8,115.0,0.006452,0.137806,0.402620,0.167844,0.188913,Female
1,1,1,0.118690,14.0,5.4,140.0,0.167742,0.202520,0.186825,0.719697,...,0.995941,15.2,5.8,152.0,0.245161,0.570656,0.980237,0.811487,0.967540,Male
2,2,2,0.203894,11.8,6.3,118.0,0.025806,0.892793,0.723152,0.725821,...,0.633806,15.4,5.7,154.0,0.258065,0.709777,0.365825,0.933026,0.646992,Male
3,3,3,0.386228,14.4,6.1,144.0,0.193548,0.024963,0.562005,0.831515,...,0.795792,13.0,6.9,130.0,0.103226,0.883454,0.399733,0.595028,0.822491,Male
4,4,4,0.564020,13.5,5.9,135.0,0.135484,0.415389,0.063705,0.211584,...,0.915185,14.2,5.8,142.0,0.180645,0.652973,0.735543,0.553571,0.532910,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2510,2510,2510,0.915744,13.9,6.4,139.0,0.161290,0.604791,0.851645,0.766959,...,0.645930,13.6,5.1,136.0,0.141935,0.443641,0.255464,0.116379,0.042459,Female
2511,2511,2511,0.820041,13.8,5.1,138.0,0.154839,0.933549,0.537350,0.918460,...,0.501251,11.9,5.4,119.0,0.032258,0.473038,0.469482,0.067636,0.210801,Female
2512,2512,2512,0.659143,12.2,5.3,122.0,0.051613,0.531385,0.456792,0.489438,...,0.701357,12.9,5.7,129.0,0.096774,0.388300,0.428321,0.322559,0.199743,Female
2513,2513,2513,0.664928,12.1,5.2,121.0,0.045161,0.538634,0.610885,0.977585,...,0.684078,13.2,6.2,132.0,0.116129,0.067981,0.167563,0.218222,0.370723,Female


In [160]:
df.columns

Index(['Unnamed: 0', 'index', 'long_hair', 'forehead_width_cm',
       'forehead_height_cm', 'forehead_width_mm',
       'forehead_width_conventional_units', 'nose_wide', 'nose_long',
       'lips_thin', 'distance_nose_to_lip_long', 'gender', 'index.1',
       'long_hair.1', 'forehead_width_cm.1', 'forehead_height_cm.1',
       'forehead_width_mm.1', 'forehead_width_conventional_units.1',
       'nose_wide.1', 'nose_long.1', 'lips_thin.1',
       'distance_nose_to_lip_long.1', 'gender.1'],
      dtype='object')

In [161]:
df["gender"].isna().sum()

1001

In [162]:
df["gender.1"].isna().sum()

29

In [163]:
df["gender"][1000]

nan

In [164]:
df = df.drop(["Unnamed: 0"], axis = 1)
df.columns

Index(['index', 'long_hair', 'forehead_width_cm', 'forehead_height_cm',
       'forehead_width_mm', 'forehead_width_conventional_units', 'nose_wide',
       'nose_long', 'lips_thin', 'distance_nose_to_lip_long', 'gender',
       'index.1', 'long_hair.1', 'forehead_width_cm.1', 'forehead_height_cm.1',
       'forehead_width_mm.1', 'forehead_width_conventional_units.1',
       'nose_wide.1', 'nose_long.1', 'lips_thin.1',
       'distance_nose_to_lip_long.1', 'gender.1'],
      dtype='object')

In [165]:
cols = df.columns[:len(df.columns) // 2]
cols

Index(['index', 'long_hair', 'forehead_width_cm', 'forehead_height_cm',
       'forehead_width_mm', 'forehead_width_conventional_units', 'nose_wide',
       'nose_long', 'lips_thin', 'distance_nose_to_lip_long', 'gender'],
      dtype='object')

In [166]:
df1 = df[cols]
df1

,index,long_hair,forehead_width_cm,forehead_height_cm,forehead_width_mm,forehead_width_conventional_units,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,0,0.704275,11.8,6.1,118.0,0.025806,0.895614,0.477485,0.841261,0.902628,NaN
1,1,0.118690,14.0,5.4,140.0,0.167742,0.202520,0.186825,0.719697,0.309122,NaN
2,2,0.203894,11.8,6.3,118.0,0.025806,0.892793,0.723152,0.725821,0.720640,NaN
3,3,0.386228,14.4,6.1,144.0,0.193548,0.024963,0.562005,0.831515,0.770379,NaN
4,4,0.564020,13.5,5.9,135.0,0.135484,0.415389,0.063705,0.211584,0.157217,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2510,2510,0.915744,13.9,6.4,139.0,0.161290,0.604791,0.851645,0.766959,0.763820,Male
2511,2511,0.820041,13.8,5.1,138.0,0.154839,0.933549,0.537350,0.918460,0.938982,Male
2512,2512,0.659143,12.2,5.3,122.0,0.051613,0.531385,0.456792,0.489438,0.457138,Female
2513,2513,0.664928,12.1,5.2,121.0,0.045161,0.538634,0.610885,0.977585,0.942940,Male


In [167]:
df2 = df[[c+".1" for c in cols]].rename(columns = {c+".1": c for c in cols})
df2

,index,long_hair,forehead_width_cm,forehead_height_cm,forehead_width_mm,forehead_width_conventional_units,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,2515,0.818018,11.5,5.8,115.0,0.006452,0.137806,0.402620,0.167844,0.188913,Female
1,2516,0.995941,15.2,5.8,152.0,0.245161,0.570656,0.980237,0.811487,0.967540,Male
2,2517,0.633806,15.4,5.7,154.0,0.258065,0.709777,0.365825,0.933026,0.646992,Male
3,2518,0.795792,13.0,6.9,130.0,0.103226,0.883454,0.399733,0.595028,0.822491,Male
4,2519,0.915185,14.2,5.8,142.0,0.180645,0.652973,0.735543,0.553571,0.532910,Male
...,...,...,...,...,...,...,...,...,...,...,...
2510,4996,0.645930,13.6,5.1,136.0,0.141935,0.443641,0.255464,0.116379,0.042459,Female
2511,4997,0.501251,11.9,5.4,119.0,0.032258,0.473038,0.469482,0.067636,0.210801,Female
2512,4998,0.701357,12.9,5.7,129.0,0.096774,0.388300,0.428321,0.322559,0.199743,Female
2513,4999,0.684078,13.2,6.2,132.0,0.116129,0.067981,0.167563,0.218222,0.370723,Female


In [168]:
test = df1[:1001]
test

,index,long_hair,forehead_width_cm,forehead_height_cm,forehead_width_mm,forehead_width_conventional_units,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,0,0.704275,11.8,6.1,118.0,0.025806,0.895614,0.477485,0.841261,0.902628,NaN
1,1,0.118690,14.0,5.4,140.0,0.167742,0.202520,0.186825,0.719697,0.309122,NaN
2,2,0.203894,11.8,6.3,118.0,0.025806,0.892793,0.723152,0.725821,0.720640,NaN
3,3,0.386228,14.4,6.1,144.0,0.193548,0.024963,0.562005,0.831515,0.770379,NaN
4,4,0.564020,13.5,5.9,135.0,0.135484,0.415389,0.063705,0.211584,0.157217,NaN
...,...,...,...,...,...,...,...,...,...,...,...
996,996,0.099102,12.7,6.0,127.0,0.083871,0.101308,0.140342,0.301002,0.863584,NaN
997,997,0.806916,14.1,5.3,141.0,0.174194,0.356022,0.237029,0.273078,0.492458,NaN
998,998,0.830750,12.1,6.2,121.0,0.045161,0.901324,0.743157,0.573616,0.486314,NaN
999,999,0.908703,14.9,6.0,149.0,0.225806,0.608539,0.886700,0.644713,0.817371,NaN


In [169]:
train = df1[1001:].append(df2)
train

,index,long_hair,forehead_width_cm,forehead_height_cm,forehead_width_mm,forehead_width_conventional_units,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
1001,1001,0.777547,13.4,6.1,134.0,0.129032,0.588097,0.803634,0.918398,0.799035,Male
1002,1002,0.469160,13.5,6.4,135.0,0.135484,0.649760,0.981002,0.610089,0.926058,Male
1003,1003,0.908917,13.4,5.4,134.0,0.129032,0.875969,0.243220,0.290800,0.159580,Female
1004,1004,0.612046,14.1,6.3,141.0,0.174194,0.407458,0.180851,0.389000,0.237359,Female
1005,1005,0.725305,12.5,5.2,125.0,0.070968,0.195681,0.442202,0.072340,0.067359,Female
...,...,...,...,...,...,...,...,...,...,...,...
2510,4996,0.645930,13.6,5.1,136.0,0.141935,0.443641,0.255464,0.116379,0.042459,Female
2511,4997,0.501251,11.9,5.4,119.0,0.032258,0.473038,0.469482,0.067636,0.210801,Female
2512,4998,0.701357,12.9,5.7,129.0,0.096774,0.388300,0.428321,0.322559,0.199743,Female
2513,4999,0.684078,13.2,6.2,132.0,0.116129,0.067981,0.167563,0.218222,0.370723,Female


In [170]:
train["gender"] = train["gender"].apply(lambda x: 1 if x == "Male" else 0)
train = train.dropna()

In [171]:
train

,index,long_hair,forehead_width_cm,forehead_height_cm,forehead_width_mm,forehead_width_conventional_units,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
1001,1001,0.777547,13.4,6.1,134.0,0.129032,0.588097,0.803634,0.918398,0.799035,1
1002,1002,0.469160,13.5,6.4,135.0,0.135484,0.649760,0.981002,0.610089,0.926058,1
1003,1003,0.908917,13.4,5.4,134.0,0.129032,0.875969,0.243220,0.290800,0.159580,0
1004,1004,0.612046,14.1,6.3,141.0,0.174194,0.407458,0.180851,0.389000,0.237359,0
1005,1005,0.725305,12.5,5.2,125.0,0.070968,0.195681,0.442202,0.072340,0.067359,0
...,...,...,...,...,...,...,...,...,...,...,...
2510,4996,0.645930,13.6,5.1,136.0,0.141935,0.443641,0.255464,0.116379,0.042459,0
2511,4997,0.501251,11.9,5.4,119.0,0.032258,0.473038,0.469482,0.067636,0.210801,0
2512,4998,0.701357,12.9,5.7,129.0,0.096774,0.388300,0.428321,0.322559,0.199743,0
2513,4999,0.684078,13.2,6.2,132.0,0.116129,0.067981,0.167563,0.218222,0.370723,0


In [172]:
test = test.drop("gender", axis = 1)

In [173]:
train.isna().sum().sum(), test.isna().sum().sum()

(0, 0)

In [174]:
def prepare(df):
    def f(x):
        return 1 if x >= 0.5 else 0
    df = df.drop(["index", "forehead_width_mm", "forehead_width_conventional_units"], axis = 1)
    for i in ['nose_wide', 'nose_long', 'lips_thin', 'distance_nose_to_lip_long', 'long_hair']:
        df[i] = df[i].apply(f)
    return df

In [175]:
train = prepare(train)
test = prepare(test)

In [176]:
train.corr()

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
long_hair,1.000000,-0.001115,-0.019623,0.009603,0.021099,0.019506,-0.018953,-0.002646
forehead_width_cm,-0.001115,1.000000,0.090493,0.257518,0.266849,0.256778,0.255094,0.340550
forehead_height_cm,-0.019623,0.090493,1.000000,0.210151,0.190617,0.198665,0.207276,0.275512
nose_wide,0.009603,0.257518,0.210151,1.000000,0.561351,0.558405,0.568477,0.758036
nose_long,0.021099,0.266849,0.190617,0.561351,1.000000,0.564888,0.564688,0.746147
lips_thin,0.019506,0.256778,0.198665,0.558405,0.564888,1.000000,0.567979,0.746547
distance_nose_to_lip_long,-0.018953,0.255094,0.207276,0.568477,0.564688,0.567979,1.000000,0.756500
gender,-0.002646,0.340550,0.275512,0.758036,0.746147,0.746547,0.756500,1.000000


In [177]:
x_train = train.drop("gender", axis = 1)
y_train = train.gender

In [178]:
model = LR()
model.fit(x_train, y_train)
pred = model.predict(test)
pred

array([1, 0, 1, ..., 1, 1, 1], dtype=int64)

In [179]:
pred = pd.Series(map(lambda x: "Male" if x == 1 else "Female", pred))
pred

0         Male
1       Female
2         Male
3         Male
4       Female
         ...  
996     Female
997     Female
998       Male
999       Male
1000      Male
Length: 1001, dtype: object

In [180]:
pred.to_csv("out2.csv", header = False, index = False)